# Feature Extraction Pipeline

This notebook extracts various features from plant images for disease detection and analysis.

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.applications import VGG16, ResNet50, EfficientNetB0
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess
from skimage.feature import local_binary_pattern, graycomatrix, graycoprops
from skimage.measure import label, regionprops
from skimage import filters, exposure
import pickle
from tqdm import tqdm
import json

# Set paths
PROJECT_ROOT = Path('/Users/debabratapattnayak/web-dev/greencast')
PROCESSED_DATA_PATH = PROJECT_ROOT / 'processed_data'
FEATURES_PATH = PROJECT_ROOT / 'features'
FEATURES_PATH.mkdir(parents=True, exist_ok=True)

## Traditional Computer Vision Features

In [5]:
class TraditionalFeatureExtractor:
    """Extract traditional computer vision features from images"""
    
    def __init__(self):
        self.feature_names = []
    
    def extract_color_features(self, image):
        """Extract color-based features"""
        features = []
        
        # Convert to different color spaces
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        
        # RGB statistics
        for i, channel in enumerate(['R', 'G', 'B']):
            channel_data = image[:, :, i]
            features.extend([
                np.mean(channel_data),
                np.std(channel_data),
                np.median(channel_data),
                np.percentile(channel_data, 25),
                np.percentile(channel_data, 75)
            ])
        
        # HSV statistics
        for i, channel in enumerate(['H', 'S', 'V']):
            channel_data = hsv[:, :, i]
            features.extend([
                np.mean(channel_data),
                np.std(channel_data)
            ])
        
        # Color ratios
        features.extend([
            np.mean(image[:, :, 1]) / (np.mean(image[:, :, 0]) + 1e-8),  # G/R ratio
            np.mean(image[:, :, 2]) / (np.mean(image[:, :, 1]) + 1e-8),  # B/G ratio
            np.mean(image[:, :, 0]) / (np.mean(image[:, :, 2]) + 1e-8),  # R/B ratio
        ])
        
        return features
    
    def extract_texture_features(self, image):
        """Extract texture-based features"""
        features = []
        
        # Convert to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
        # Local Binary Pattern
        radius = 3
        n_points = 8 * radius
        lbp = local_binary_pattern(gray, n_points, radius, method='uniform')
        lbp_hist, _ = np.histogram(lbp.ravel(), bins=n_points + 2, 
                                  range=(0, n_points + 2), density=True)
        features.extend(lbp_hist)
        
        # Gray Level Co-occurrence Matrix (GLCM)
        # Normalize gray image to 0-255 range for GLCM
        gray_norm = ((gray - gray.min()) / (gray.max() - gray.min()) * 255).astype(np.uint8)
        
        # Calculate GLCM for different angles
        distances = [1, 2]
        angles = [0, 45, 90, 135]
        
        for distance in distances:
            for angle in angles:
                glcm = graycomatrix(gray_norm, [distance], [np.radians(angle)], 
                                  levels=256, symmetric=True, normed=True)
                
                # Extract GLCM properties
                features.extend([
                    graycoprops(glcm, 'contrast')[0, 0],
                    graycoprops(glcm, 'dissimilarity')[0, 0],
                    graycoprops(glcm, 'homogeneity')[0, 0],
                    graycoprops(glcm, 'energy')[0, 0],
                    graycoprops(glcm, 'correlation')[0, 0]
                ])
        
        return features
    
    def extract_shape_features(self, image):
        """Extract shape-based features"""
        features = []
        
        # Convert to grayscale and threshold
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
        # Apply Otsu's thresholding
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Find contours
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            # Get the largest contour (assuming it's the main object)
            largest_contour = max(contours, key=cv2.contourArea)
            
            # Calculate shape features
            area = cv2.contourArea(largest_contour)
            perimeter = cv2.arcLength(largest_contour, True)
            
            if perimeter > 0:
                # Circularity
                circularity = 4 * np.pi * area / (perimeter * perimeter)
                features.append(circularity)
            else:
                features.append(0)
            
            # Aspect ratio
            x, y, w, h = cv2.boundingRect(largest_contour)
            aspect_ratio = float(w) / h if h > 0 else 0
            features.append(aspect_ratio)
            
            # Extent (ratio of contour area to bounding rectangle area)
            rect_area = w * h
            extent = float(area) / rect_area if rect_area > 0 else 0
            features.append(extent)
            
            # Solidity (ratio of contour area to convex hull area)
            hull = cv2.convexHull(largest_contour)
            hull_area = cv2.contourArea(hull)
            solidity = float(area) / hull_area if hull_area > 0 else 0
            features.append(solidity)
            
        else:
            # No contours found, add default values
            features.extend([0, 0, 0, 0])
        
        return features
    
    def extract_all_features(self, image):
        """Extract all traditional features from an image"""
        features = []
        
        # Extract different types of features
        color_features = self.extract_color_features(image)
        texture_features = self.extract_texture_features(image)
        shape_features = self.extract_shape_features(image)
        
        features.extend(color_features)
        features.extend(texture_features)
        features.extend(shape_features)
        
        return np.array(features)

# Initialize feature extractor
traditional_extractor = TraditionalFeatureExtractor()
print("Traditional feature extractor initialized!")

Traditional feature extractor initialized!


## Deep Learning Feature Extraction

In [6]:
# LIGHTWEIGHT FEATURE EXTRACTOR - OPTIMIZED FOR DISK SPACE
import sys
sys.path.append('/Users/debabratapattnayak/web-dev/greencast')
from lightweight_feature_extractor import LightweightFeatureExtractor

# Enhanced DeepFeatureExtractor using lightweight backend
class DeepFeatureExtractor:
    def __init__(self):
        print("Initializing lightweight feature extractor...")
        try:
            self.extractor = LightweightFeatureExtractor(use_deep_features=True)
            print("✅ Deep feature extractor initialized with VGG16!")
        except Exception as e:
            print(f"⚠️ Deep features failed: {e}")
            self.extractor = LightweightFeatureExtractor(use_deep_features=False)
            print("✅ Basic feature extractor initialized!")
    
    def extract_all_features(self, image_path):
        return self.extractor.extract_all_features(image_path)
    
    def extract_vgg16_features(self, image_path):
        import cv2
        image = cv2.imread(str(image_path))
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            return self.extractor.extract_deep_features(image)
        return []
    
    def extract_resnet50_features(self, image_path):
        return self.extract_vgg16_features(image_path)
    
    def extract_efficientnet_features(self, image_path):
        return self.extract_vgg16_features(image_path)


## Feature Extraction Pipeline

In [7]:
def extract_features_from_dataset(dataset_path, output_path, sample_size=None):
    """Extract features from entire dataset"""
    
    # Collect all image paths and labels
    image_paths = []
    labels = []
    
    for class_name in os.listdir(dataset_path):
        class_path = os.path.join(dataset_path, class_name)
        if os.path.isdir(class_path):
            for img_file in os.listdir(class_path):
                if img_file.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                    image_paths.append(os.path.join(class_path, img_file))
                    labels.append(class_name)
    
    # Sample if requested
    if sample_size and len(image_paths) > sample_size:
        indices = np.random.choice(len(image_paths), sample_size, replace=False)
        image_paths = [image_paths[i] for i in indices]
        labels = [labels[i] for i in indices]
    
    print(f"Extracting features from {len(image_paths)} images...")
    
    # Initialize feature storage
    traditional_features = []
    deep_features = {'vgg16': [], 'resnet50': [], 'efficientnet': []}
    valid_labels = []
    valid_paths = []
    
    # Extract features
    for i, img_path in enumerate(tqdm(image_paths, desc="Extracting features")):
        try:
            # Load image
            img = cv2.imread(img_path)
            if img is None:
                continue
            
            # Convert to RGB
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            # Resize to standard size
            img_resized = cv2.resize(img_rgb, (224, 224))
            
            # Extract traditional features
            trad_feat = traditional_extractor.extract_all_features(img_resized)
            traditional_features.append(trad_feat)
            
            # Extract deep features
            deep_feat = deep_extractor.extract_all_model_features(img_resized)
            for model_name, features in deep_feat.items():
                deep_features[model_name].append(features)
            
            valid_labels.append(labels[i])
            valid_paths.append(img_path)
            
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
            continue
    
    # Convert to numpy arrays
    traditional_features = np.array(traditional_features)
    for model_name in deep_features.keys():
        deep_features[model_name] = np.array(deep_features[model_name])
    
    # Save features
    output_path = Path(output_path)
    output_path.mkdir(parents=True, exist_ok=True)
    
    # Save traditional features
    with open(output_path / 'traditional_features.pkl', 'wb') as f:
        pickle.dump({
            'features': traditional_features,
            'labels': valid_labels,
            'paths': valid_paths
        }, f)
    
    # Save deep features
    for model_name, features in deep_features.items():
        with open(output_path / f'{model_name}_features.pkl', 'wb') as f:
            pickle.dump({
                'features': features,
                'labels': valid_labels,
                'paths': valid_paths
            }, f)
    
    # Save metadata
    metadata = {
        'num_images': len(valid_labels),
        'num_classes': len(set(valid_labels)),
        'classes': list(set(valid_labels)),
        'traditional_feature_dim': traditional_features.shape[1],
        'deep_feature_dims': {name: feat.shape[1] for name, feat in deep_features.items()}
    }
    
    with open(output_path / 'metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\nFeature extraction complete!")
    print(f"Processed {len(valid_labels)} images")
    print(f"Traditional features shape: {traditional_features.shape}")
    for model_name, features in deep_features.items():
        print(f"{model_name} features shape: {features.shape}")
    
    return metadata

# Extract features from processed dataset
processed_train_path = PROCESSED_DATA_PATH / 'plantvillage_color' / 'train'
if processed_train_path.exists():
    print("Extracting features from training set...")
    train_metadata = extract_features_from_dataset(
        processed_train_path,
        FEATURES_PATH / 'train',
        sample_size=1000  # Sample for demonstration
    )
else:
    print("Processed training data not found. Please run the preprocessing notebook first.")

Extracting features from training set...
Extracting features from 1000 images...


Extracting features:   1%|          | 7/1000 [00:00<00:27, 36.34it/s]

Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Grape___Esca_(Black_Measles)/3a4f2681-0ad1-4ad1-a05b-005592733970___FAM_B.Msls 1037.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/391bd206-4341-4404-9904-945b0ce617a1___YLCV_NREC 2584.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Cherry_(including_sour)___Powdery_mildew/886e024a-ae28-4692-96dd-f8c7a5c38c19___FREC_Pwd.M 4759.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Soybean___healthy/a6ef7cfa-e411-4fff-945c-e658860318e8___RS_HL 2943.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillag

Extracting features:   2%|▏         | 20/1000 [00:00<00:19, 50.24it/s]

Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Pepper,_bell___Bacterial_spot/dd1be385-6753-4a84-ab24-bf4b9ede3950___NREC_B.Spot 9198.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Soybean___healthy/20ced3da-9b1e-4af7-94c1-13c9f2525769___RS_HL 5596.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Tomato___Spider_mites Two-spotted_spider_mite/35a278b4-e474-4a62-99d5-80bb10639193___Com.G_SpM_FL 1368.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Tomato___Tomato_Yellow_Leaf_Curl_Virus/7eb45688-09bb-43bf-a5f6-38e57e32eeed___YLCV_NREC 2003.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/pl

...

Extracting features: 100%|█████████▉| 996/1000 [00:17<00:00, 54.58it/s]

Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Apple___Cedar_apple_rust/affae282-693e-4078-b037-de7037ecdcdc___FREC_C.Rust 9909.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Cherry_(including_sour)___Powdery_mildew/3241ea49-4272-490b-95d5-6797a3aea739___FREC_Pwd.M 0408.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Tomato___Leaf_Mold/73f77cb0-f4ee-4500-ac3f-2cd6c79585ed___Crnl_L.Mold 7000.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Grape___Black_rot/c01ce5a9-a37d-4698-be6a-335868371a54___FAM_B.Rot 3594.JPG: name 'deep_extractor' is not defined
Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Tom

Extracting features: 100%|██████████| 1000/1000 [00:17<00:00, 55.83it/s]


Error processing /Users/debabratapattnayak/web-dev/greencast/processed_data/plantvillage_color/train/Pepper,_bell___Bacterial_spot/65c3a8f9-9631-4b86-9ae3-28116c869247___JR_B.Spot 9005.JPG: name 'deep_extractor' is not defined


IndexError: tuple index out of range